In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [74]:
df = pd.read_csv("LinkedIn Scraper Results Batch 1.csv")
df = df[df.columns[:-13]]
df = df.drop(columns = ["Email", "Assigned To", "LinkedIn URLs"])
df = df.fillna("0")
print(df.shape)
df.head(10)

(1311, 7)


,Name,Headline,Company,Experiences,Education,Yes/No/Maybe,Notes
0,Andrea Gewirtz,Health & Wealth Strategist at WellBeing & Worth,WellbeingandWorth,"Founder @ WellbeingandWorth, Oct 2019 – Presen...","The Johns Hopkins University, 1986 – 2001\n Pe...",No,0
1,Muskaan N.,Founder at Aromatherapy University,Aromatherapy University,Founder @ Aromatherapy University\n Self-empl...,"Master's degree @ Harvard University, 2019 – 2...",No,0
2,"Michael Morton, CFP®, ChFC®",Financial Advisor for Entrepreneurs and Tech P...,Morton Financial Advice,Founder @ Morton Financial Advice\n Self-empl...,"BA @ Swarthmore College, 1993 – 1997\n MS @ Th...",No,0
3,Alvin Alexander,Founder & CEO at Orion Applied Science & Techn...,"Orion Applied Science & Technology, LLC",Founder & CEO @ Orion Applied Science & Techno...,"MS @ The Johns Hopkins University, 2010 – 2014...",No,0
4,Tracey Miller,"Women's Wellness Exchange, Adventures Within, LLC",Women's Wellness Exchange,"Founder @ Women's Wellness Exchange, Dec 2020 ...",Master’s Degree @ The Johns Hopkins University...,No,0
5,Mark Nicastre,Founder at Fitler Strategies,Fitler Strategies,"Founder @ Fitler Strategies \n Self-employed,...",MA @ The Johns Hopkins University\n BA @ The J...,No,0
6,Carrie Varoquiers,"Chief Philanthropy Officer, Workday & Founder,...",0,"Chief Philanthropy Officer @ Workday, 2021 – P...","BA @ University of California, Berkeley\n MPH ...",No,0
7,Alexander Velikanov,Co-Founder and CTO at Pearl 👩‍💻,Pearl,"Co-Founder and CTO @ Pearl\n Full-time, Sep 2...",Bachelor of Science - BS @ University of Calif...,No,Already in Trello
8,Warren Strauss,"President at Karna, LLC","Karna, LLC","President @ Karna, LLC, Sep 2019 – Present\n F...","Sc.M. @ The Johns Hopkins University, 1990 – 1...",No,0
9,Robert Kviby,Chairman @ Staccs | Founder of The Growth Co.,The Growth Co.,"Founder @ The Growth Co., Jan 2018 – Present\n...","Duke University, 2015 – 2015\n The Johns Hopki...",No,0


In [75]:
# making outcome binary (no = 0, maybe = 1, yes = 2)

df["Outcome"] = np.zeros(df.shape[0])

for row in df.index:
    if df["Yes/No/Maybe"][row] == "No":
        df["Outcome"][row] = 0
    if df["Yes/No/Maybe"][row] == "Maybe":
        df["Outcome"][row] = 1
    if df["Yes/No/Maybe"][row] == "Yes":
        df["Outcome"][row] = 2


<ipython-input-75-9983109f3e6f>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"][row] = 0
<ipython-input-75-9983109f3e6f>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"][row] = 1
<ipython-input-75-9983109f3e6f>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Outcome"][row] = 2


In [76]:
df["Outcome"].value_counts()

0.0    1218
1.0      56
2.0      37
Name: Outcome, dtype: int64

In [77]:
# trying bag of words vectorization

from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5, norm='l2', encoding='latin-1', ngram_range=(1, 2), stop_words='english')
features = tfidf.fit_transform(df.Name).toarray()
labels = df.Outcome
features.shape



(1311, 32)

In [82]:
# multinomial naive bayes (x is ONLY headline, y is outcome 0/1/2)

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB

X = df.drop(columns = "Outcome")
y = df["Outcome"]

X_train, X_test, y_train, y_test = train_test_split(X["Experiences"], y, test_size = 0.25, random_state = 42)

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_train)

tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)

clf = MultinomialNB().fit(X_train_tfidf, y_train)

In [84]:
X_test_transformed = count_vect.transform(X_test)

#clf.predict(count_vect.transform(["Health tech startup harvard yale princeton ceo money"]))
clf.predict(X_test_transformed)

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.